<a href="https://colab.research.google.com/github/anruki/Rubik_Encription/blob/main/encription_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de cifrado empleando las permutaciones del cubo de Rubik

## Importación de las librerías de python necesarias

In [12]:
import numpy as np
import secrets

La biblioteca `numpy` de python proporciona estructuras como arrays, y funciones para operar con ellos.

- Ejemplo de uso:

In [13]:
# Creación de dos matrices utilizando NumPy
matriz_a = np.array([[1, 2, 3], [4, 5, 6]])
matriz_b = np.array([[7, 8, 9], [10, 11, 12]])

# Multiplicación matricial (producto punto)
producto_punto = np.dot(matriz_a, matriz_b.T)

La biblioteca `secrets` de python permite generar números aleatorios de forma segura (TODO: explicar aquí por qué)

- Ejemplo de uso:

In [ ]:
# Generación de un número aleatorio seguro en el rango especificado
num_aleatorio = secrets.randbelow(100)  # Genera un número aleatorio entre 0 y 99 (inclusive)
print("Número aleatorio:", num_aleatorio)

# Generación de números aleatorios con un número específico de bits
num_aleatorio_bits = secrets.randbits(16)  # Genera un número aleatorio de 16 bits
print("Número aleatorio (16 bits):", num_aleatorio_bits)

Número aleatorio: 29
Número aleatorio (16 bits): 27420


### Transformación binaria

Los ordenadores manejan expresiones en binario y no en el alfabeto. Para simular el proceso de encriptación en un ordenador, crearemos una función que transforme el mensaje a binario.

In [1]:
def transformacion_binaria(mensaje):
    binarios = [format(ord(letra), '08b') for letra in mensaje]
    return binarios


Para representar 26 letras del alfabeto inglés, se necesita un número entero de bits que sea igual o mayor al logaritmo en base 2 de 26 (log₂(26)).

El logaritmo en base 2 de un número N nos dice cuántos bits necesitaríamos para representar N valores distintos. En este caso, el logaritmo en base 2 de 26 es aproximadamente 4.7. Dado que necesitamos un número entero de bits, redondearemos hacia arriba a 5. Por lo tanto, necesitarías al menos 5 bits para representar 26 letras del alfabeto inglés de forma única.

Siempre es recomendable redondear hacia arriba para asegurarnos de tener suficientes bits para representar todos los elementos de manera única. En este caso, redondeamos hacia arriba a 5 bits para representar 26 letras del alfabeto inglés.

Y su función inversa para posteriormente poder decodificar el mensaje:

In [6]:
def transformacion_caracteres(lista_binaria):
    mensaje = ''
    for binario in lista_binaria:
        caracter = chr(int(binario, 2))
        mensaje += caracter
    return mensaje

Probamos con un mensaje de ejemplo:

- Transformación a formato binario:

In [4]:
x = transformacion_binaria('ejemplo')
x = np.array(x)
print(x)

['01100101' '01101010' '01100101' '01101101' '01110000' '01101100'
 '01101111']


- Transformación de formato binario a caracteres:

In [7]:
y = transformacion_caracteres(x)
print(y)

ejemplo


## Transformación del mensaje a numérico

En nuestro caso, para ver fácilmente la operación del proceso de encriptación, vamos a pasar el mensaje en caracteres a un mensaje compuesto por números, cada letra del abecedario es representada por un valor numérico. Por ejemplo:

A: 0
B: 1
C:2

TODO: cifrado César aquí

In [10]:
def caracteres_a_numeros(texto):
    def caracter_a_numero(caracter):
        if not caracter.isalpha():
            return None  # Retornar None si no es un carácter alfabético
        return ord(caracter.lower()) - ord('a')

    # Convertir cada carácter del texto a su equivalente numérico y guardarlos en el array
    numeros = []
    for caracter in texto:
        numero = caracter_a_numero(caracter)
        if numero is not None:
            numeros.append(numero)

    return np.array(numeros)  # Convertir la lista a un array NumPy

Probamos un mensaje de ejemplo y llamamos a la función anterior.

In [74]:
# Cadena de texto de ejemplo
texto = 'abcdef'

# Llamar a la función con el texto dado
resultado = caracteres_a_numeros(texto)

print("Números obtenidos:", resultado)

Números obtenidos: [0 1 2 3 4 5]


## Creación de la función de encriptación simétrica

El mensaje en binario se le aplica a una matriz que representa las permutaciones del cubo de Rubik. Concretamente es una matriz con 2x3 dimensiones, para representar las 2 caras permutables del cubo y los 3 ejes `[x,y,z]` sobre los que se puede rotar cada cara. Para que la matriz sea cuadrada (necesario para la encriptación simétrica) se le añade una fila de unos.

Al tratarse de una estructura cúbica, trabajamos en modulo 4, ya que cada permutación, si se repite el mismo movimiento 4 veces, se vuelve a la posición inicial.

**Generación de la matriz en el cuerpo módulo 4**

Se utiliza:
 - La función `randbelow()` de la biblioteca `secrets` para generar números aleatorios **módulo 4**, es decir de 0 a 3.

- La función `.array()` de la biblioteca `numpy` para obtener una matriz de `3 filas` y `2 columnas`

La notación de llamada de la función secrets.randbelow(4) y np.array() sigue la convención de utilizar el punto para acceder a los métodos y atributos dentro de un módulo o biblioteca en Python.

In [75]:
def rubik_matrix():
    # Función que calcula una matriz de permutaciones aleatorias (con una fila de ceros agregada para que sea cuadrada)

    # Definición del número de filas y columnas
    filas = 3
    columnas = 3
    # Inicialización de la matriz
    matriz_permutaciones = []

    # Crear la matriz con números aleatorios
    matriz_permutaciones = np.array([[secrets.randbelow(4) for _ in range(columnas)] for _ in range(filas-1)])

    # Agregar una fila de unos al final de la matriz
    fila_ceros = np.ones((1, columnas), dtype=int)
    matriz_permutaciones = np.append(matriz_permutaciones, fila_ceros, axis=0)

    return matriz_permutaciones

In [76]:
# Crear la matriz con números aleatorios
matriz_permutaciones = rubik_matrix()

# Asegurar que la matriz tenga inversa (determinante no nulo)
while np.linalg.det(matriz_permutaciones) == 0:
    matriz_permutaciones = rubik_matrix()

print(matriz_permutaciones)

[[1 2 1]
 [0 2 2]
 [1 1 1]]


**Multiplicación de la matriz por el vector del mensaje**

TODO: si el mensahe es impar, añadir -1

En un cifrado simétrico como este, la matriz (nuestra `llave privada`) tomará valores diferentes cada vez que se encripte un mensaje para una gestión de mensajes más segura.

 Sin embargo, las dimensiones de la matriz de permutaciones siempre son las mismas, y no tienen por qué coincidir con las dimensiones del mensaje a encriptar, es por ello, que se ha de 'trocear' o hacer 'slicing' del mensaje para que independientemente de sus dimensiones, pueda ser multiplicado matricialmente con la matriz de permutaciones.

In [77]:
resultado

array([0, 1, 2, 3, 4, 5])

In [85]:
# cifrado_num = np.dot(matriz_permutaciones, resultado)
n = resultado.size
# Si el tamaño del mensaje es impar, añado un -1 al final
while n%3 != 0:
  np.append(resultado,-1)
m = n//3
resultado_2 = resultado.reshape(m, 3)
print(resultado_2)
# hago slice del array mensaje
cifrado_num = []
for i in range(m):
  cifrado_num = cifrado_num + [np.dot(matriz_permutaciones, resultado_2[i])]

[[0 1 2]
 [3 4 5]]


In [80]:
cifrado_num = cifrado_num.astype(int)

In [88]:
cifrado_num

[array([4, 6, 3]), array([16, 18, 12])]

Pasamos a carácter de nuevo.
TODO: si hay -1 = ''


In [91]:
def numeros_a_caracteres(numeros):
    def numero_a_caracter(numero):
        if numero < 0 or numero >= 26:
            return None  # Retornar None si el número está fuera del rango válido
        return chr(numero + ord('a'))

    # Convertir cada número en el array a su equivalente alfabético y guardarlos en una lista
    caracteres = []
    for numero in numeros:
        caracter = numero_a_caracter(numero)
        if caracter is not None:
            caracteres.append(caracter)

    # Unir los caracteres en una cadena de texto
    return ''.join(caracteres)
cifrado_char = numeros_a_caracteres(np.concatenate(cifrado_num))

In [92]:
cifrado_char

'egdqsm'

Para descifrar el mensaje, calculamos la inversa de la matriz transformación:

In [98]:
# Calcular la pseudoinversa
inversa = np.linalg.inv(matriz_permutaciones)
print(inversa)

[[ 0.  -0.5  1. ]
 [ 1.   0.  -1. ]
 [-1.   0.5  1. ]]


In [99]:
descifrado_num = []
for i in range(m):
  descifrado_num = descifrado_num + [np.dot(inversa, cifrado_num[i])]
print(descifrado_num)

[array([0., 1., 2.]), array([3., 4., 5.])]


In [101]:
mensaje = []
for i in range(m):
  mensaje = np.append(mensaje,numeros_a_caracteres(descifrado_num[i].astype(int)))
print(mensaje)

['abc' 'def']


Y así obtenemos el mensaje inicial.